In [1]:
from aeroTesting import AeroRepeatability, Probe

In [ ]:
# user sequence:
golden_probe_path = input('Path to golden probe data: ')
uut_probe_path = input('Path to uut probe data: ')
AeroRepeatability(golden_probe_path, uut_probe_path)

In [2]:
golden_probe_path = '/home/stoker/aero_analysis/A0251-50kn'
uut_probe_path = '/home/stoker/aero_analysis/SN003-50kn'

In [ ]:
uut_probe.data_path

In [3]:
AeroRepeatability(golden_probe_path, uut_probe_path)

Known headers:
aerosonic_default
test1
wrong
Choose an existing header or type NO:aerosonic_default
{'2': 'tunnel_ref_total_pressure', '10': 'angle', '1': 'time', '5': 'uut_total_pressure', '9': 'tempF', '3': 'tunnel_ref_static_pressure', '0': 'date', '6': 'uut_static_pressure1', '7': 'uut_static_pressure2'}


/home/stoker/aero_analysis/aeroTesting.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.loc['angle'] = df['angle'].apply(self.normalize_angle)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [4]:
golden_probe = Probe(golden_probe_path)


Known headers:
aerosonic_default
wrong
test1
Choose an existing header or type NO:aerosonic_default
{'0': 'date', '1': 'time', '5': 'uut_total_pressure', '10': 'angle', '9': 'tempF', '2': 'tunnel_ref_total_pressure', '3': 'tunnel_ref_static_pressure', '6': 'uut_static_pressure1', '7': 'uut_static_pressure2'}


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
uut_probe = Probe(uut_probe_path)

Known headers:
wrong
test1
aerosonic_default
Choose an existing header or type NO:aerosonic_default
{'0': 'date', '1': 'time', '6': 'uut_static_pressure1', '2': 'tunnel_ref_total_pressure', '5': 'uut_total_pressure', '7': 'uut_static_pressure2', '10': 'angle', '3': 'tunnel_ref_static_pressure', '9': 'tempF'}


/home/stoker/aero_analysis/aeroTesting.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['angle'] = df['angle'].apply(self.normalize_angle)


In [ ]:
uut_probe.cp

In [14]:

import pandas as pd
import os
import json
import matplotlib.pyplot as plt
from datetime import date
import ntpath
